## 引入所需套件

In [1]:
import urllib.request as req
import bs4 as bs
import urllib.request as req
import urllib.parse
import pandas as pd

## 開始爬蟲

In [2]:
def get_html_from_url(url):
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp)
    return html

In [11]:
def crawl_a_job(job_link):
    html = get_html_from_url(job_link)

    descriptions = html.find_all('div', {'class': 'job_explain'})
    '''
    ul_item = description.find('ul')
    li_items = ul_item.find_all('li')
    '''
    li_items = []
    for description in descriptions:
        ul_item = description.find('ul')
        li_items += ul_item.find_all('li')

    # 我們需要的項目（還沒爬更新日期、程式技能）
    map_dict = {
        '工作內容': 'description',
        '工作性質': 'work_type',
        '上班時段': 'work_time',
        '工作地點': 'address',

        '學歷要求': 'degree',
        '科系要求': 'department',
        '工作經驗': 'working_experience', # 暫時用文字
        '外語能力': 'qualification_bonus',

        '連絡人': 'contact_person'
    }

    job_dict = {}

    # 還是保留個空值
    for key, value in map_dict.items():
        job_dict[value] = ''

    for li_item in li_items:
        # 先爬取這個項目的副標題
        left_title = li_item.find('span', {'class': 'left_title'})
        if left_title is None:
            continue
        left_title_text = left_title.get_text()
        # 再一一核對職缺說明有沒有這一項
        for key, value in map_dict.items():
            if key in left_title_text:
                right_main_text = li_item.find('span', {'class': 'right_main'}).get_text()
                job_dict[value] = right_main_text.strip()

    return job_dict

In [5]:
def crawl_all_results(html):
    job_items = html.find_all("div",{"class":"Job_opening_M"})
    print(len(job_items))

    job_list = []

    for job_item in job_items:
        print('---------------------------')
        divs = job_item.find("div",{"class":"Job_opening_item_info_M"}).find_all('div')
        # 工作名稱
        title_text = divs[1].find('h5').get_text().strip()
        # 以下兩個之後會在table「company」獨立建表
        company_text = divs[2].find('h5').get_text().strip()
        # 地區
        location_text = divs[3].find('h5').get_text().strip()
        salary_text = divs[4].find('h5').get_text().strip()

        # tags: 有些有有些沒有
        tags = ''
        if len(divs) > 5:
            for div in divs[5:]:
                tags += div.get_text().strip()

        job_dict = {
            "title": title_text,
            "salary": salary_text,
            "company": company_text,
            "location": location_text,
            "tags": tags
        };
        print(job_dict)

        # 準備爬取每個職缺連結
        job_link = job_item.find('a')['href']
        job_link = 'https://www.yes123.com.tw/wk_index/' + job_link
        print(job_link)

        # 爬取頁面所需資訊（未完）
        page_dict = crawl_a_job(job_link)
        print(page_dict)

        # 結合主頁面和單一工作頁面所得資訊，加入串列中
        job_dict = job_dict | page_dict
        job_list.append(job_dict)

    return job_list

In [6]:
find_work_mode1 = "2_1011_0001_0017"
s_find_work_mode1 = "資料工程師"

# 原始 URL，含有中文
url = f'https://www.yes123.com.tw/wk_index/joblist.asp?find_work_mode1={find_work_mode1}&s_find_work_mode1={s_find_work_mode1}&order_ascend=desc&strrec=0&search_type=job&search_from=joblist'

# 將中文部分進行編碼
parsed = urllib.parse.urlsplit(url)
encoded_path = urllib.parse.quote(parsed.path)
encoded_query = urllib.parse.quote(parsed.query, safe="=&")
# 組合回完整 URL
url = urllib.parse.urlunsplit((parsed.scheme, parsed.netloc, encoded_path, encoded_query, parsed.fragment))
print(url)

https://www.yes123.com.tw/wk_index/joblist.asp?find_work_mode1=2_1011_0001_0017&s_find_work_mode1=%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB&order_ascend=desc&strrec=0&search_type=job&search_from=joblist


In [12]:
if __name__ == '__main__':
    html = get_html_from_url(url)
    job_list = crawl_all_results(html)

24
---------------------------
{'title': 'AI資料工程師(桃園)', 'salary': '薪資面議(經常性薪資達4萬元含以上)', 'company': '英業達股份有限公司', 'location': '桃園市桃園區', 'tags': ''}
https://www.yes123.com.tw/wk_index/job.asp?p_id=83151_04322046&job_id=20250509042616_21320081
{'description': '1.參與-資料安全設計-資料被安全的存取2.協助-內部系統資料串接-將AI與公司內部資料有效整合3.參與-AI資料治理-資料能有效管理與運用4.協助-新技術評估-運用新的技術提升資料管理效率', 'work_type': '全職', 'work_time': '', 'address': '桃園市桃園區大智路88號(龜山工業區)', 'degree': '大學', 'department': '資訊管理學類、資訊工程學類', 'working_experience': '2 年以上', 'qualification_bonus': '英文→聽：普通\u3000說：普通\u3000讀：普通\u3000寫：普通', 'contact_person': '人才招募部'}
---------------------------
{'title': 'AI軟體工程師 (大溪)', 'salary': '薪資面議(經常性薪資達4萬元含以上)', 'company': '英業達股份有限公司', 'location': '桃園市大溪區', 'tags': ''}
https://www.yes123.com.tw/wk_index/job.asp?p_id=83151_04322046&job_id=20241216035610_44313675
{'description': '我們正在尋找對人工智慧充滿熱情並渴望在快速發展的技術領域中發揮影響力的AI軟體工程師。加入我們的團隊，您將有機會參與前沿技術的研發和應用，推動公司的技術創新與進步。工作內容：1. 參與並追蹤最新的AI技術，提出改進方案，持續導入新AI技術，提升系統和應用場景2. 跨部門團隊合作，了解需求並轉化為AI解

## 轉換成json檔案後下載

In [ ]:
import json
fn = s_find_work_mode1+'.json'
with open(fn, 'w', encoding='utf-8') as f:
    # endure-ascii 會把所有東西轉成英文
    json.dump(job_list, f, ensure_ascii=False, indent=4)

## 轉換成dataframe

In [13]:
df = pd.DataFrame(job_list)
df

,title,salary,company,location,tags,description,work_type,work_time,address,degree,department,working_experience,qualification_bonus,contact_person
0,AI資料工程師(桃園),薪資面議(經常性薪資達4萬元含以上),英業達股份有限公司,桃園市桃園區,,1.參與-資料安全設計-資料被安全的存取2.協助-內部系統資料串接-將AI與公司內部資料有效...,全職,,桃園市桃園區大智路88號(龜山工業區),大學,資訊管理學類、資訊工程學類,2 年以上,英文→聽：普通 說：普通 讀：普通 寫：普通,人才招募部
1,AI軟體工程師 (大溪),薪資面議(經常性薪資達4萬元含以上),英業達股份有限公司,桃園市大溪區,,我們正在尋找對人工智慧充滿熱情並渴望在快速發展的技術領域中發揮影響力的AI軟體工程師。加入我...,全職,,桃園市大溪區(依公司規定),大學以上,不拘,2 年以上,英文→聽：普通 說：普通 讀：普通 寫：普通,人才招募部
2,AI工程師,"月薪 40,000 至 45,000元",快樂島股份有限公司,台北市內湖區,#免經驗\n#二度就業\n#彈性上下班,工作內容1.負責 AI/機器學習模型的開發、訓練與優化2.參與產品研發，將 AI 技術應用於...,全職,依公司規定,台北市內湖區瑞光路335號3樓(內湖科技園區),專科以上,工程相關、數學及統計相關,不拘,,黃俊頴
3,Data Scientist (士林),"月薪 55,000元以上",英業達股份有限公司,台北市士林區,,英業達數位中心為一研究與發展部門，扁平化的組織型態，涵蓋專案經理、資料科學家、人工智慧研究員...,全職,彈性,台北市士林區承德路四段166號,大學以上,電算機相關、統計學類、電機電子工程相關,不拘,英文→聽：精通 說：精通 讀：精通 寫：精通中文→聽：精通 說：精通 讀：精通 寫：精通,HR
4,AI軟體工程師 (士林),薪資面議(經常性薪資達4萬元含以上),英業達股份有限公司,台北市士林區,,我們正在尋找對人工智慧充滿熱情並渴望在快速發展的技術領域中發揮影響力的AI軟體工程師。加入我...,全職,,台北市士林區後港街66號,大學,資訊管理學類、資訊工程學類、其他數學及統計學類,1 年以上,英文→聽：普通 說：普通 讀：普通 寫：普通,HR
5,資訊部-BI系統工程師,薪資面議(經常性薪資達4萬元含以上),王品餐飲股份有限公司,台中市西區,,王品集團 誠徵【資訊部-BI系統工程師】【職責內容】1. 深入了解 BI 專案需求，進行用戶...,全職,08:30~17:30,台中市西區台灣大道2段218號26樓,大學,不拘,2 年以上,,人資發展部/招募組
6,[Server]DBA資料庫管理師(桃園),薪資面議(經常性薪資達4萬元含以上),英業達股份有限公司,桃園市桃園區,,1. 負責資料庫的安裝、配置及日常維護，確保資料庫的高效率穩定運作；2. 參與資料庫架構設計...,全職,08:00~17:00,桃園市桃園區大智路88號龜山工業區(龜山工業區),大學,資訊管理學類,不拘,英文→聽：略懂 說：略懂 讀：略懂 寫：略懂,人才招募部
7,營運部-系統設計與維護規劃師,"月薪 38,000 至 45,000元",王品餐飲股份有限公司,台中市西區,,王品集團 誠徵【營運部-系統設計與維護規劃師】工作內容：1.報表製作、數據統計和分析業務數據...,全職,08:30~17:30,台中市西區台灣大道二段218號24樓,專科以上,不拘,2 年以上,,人資發展部/招募組
8,【總公司_資訊部】管理師,薪資面議(經常性薪資達4萬元含以上),新東陽股份有限公司,台北市大安區,,1.SAP MM模組日常維護2.POS/WMS外部系統串接3.熟SAP ABAP程式設計,全職,09:00~18:00、白天班,台北市大安區忠孝東路四段289號8樓,大學以上,,2 年以上,,HR
9,總公司 Junior Data Product Manager_台北,薪資面議(經常性薪資達4萬元含以上),(家樂福)家福股份有限公司,台北市北投區,,- BI 及 AI 產品維運：使用者問題釐清排除、使用狀況及資源監測、教育訓練講授與規劃。-...,全職,09:00~18:30,台北市北投區大業路136號,大學以上,不拘,1 年以上,,人力資源部


In [14]:
df.to_csv('yes123.csv')

## 備用：如果需要使用 selenium

### 安裝所需套件

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

### 載入套件

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.headless = True
driver = webdriver.Chrome(options=chrome_options)

### 課堂範例用法

In [ ]:
driver.get("https://www.google.com")
driver.maximize_window()
# 開始使用selenium
# 常見語法：BY.TAG_NAME、BY.CLASS_NAME、BY.ID
# 尋找Chrome首頁的文字輸入框
e = driver.find_element(By.CLASS_NAME, "gLFyf")
print(e.get_attribute("class"))
# 點擊：click()，鍵盤輸入：send_keys("aaa")
e.send_keys("chiikawa")
e.send_keys(Keys.ENTER)

gLFyf
